In [2]:
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()

# temperature가 낮을수록 낮은 창의성을 갖는다.
chat = ChatOpenAI(
    temperature=0.1
)

'As of now, there are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, there is ongoing debate among astronomers regarding the classification of Pluto as a planet, as it was reclassified as a "dwarf planet" by the International Astronomical Union in 2006.'

In [3]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    # AI 설정
    SystemMessage(
        content="You are a geography expert. And you only reply in Italian."
    ),
    # 마치 AI가 답변한 것으로 가정
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(
        content="What is the distance between Mexico and Thailand. Also, what is your name?"
    )
]

chat.predict_messages(messages)

AIMessage(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')